In [2]:
# Import packages
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)


In [24]:
df_blood1_raw = pd.read_excel('2. BLOODWORK_blindedSep28.xlsx')  
df_static_raw = pd.read_excel('1. LISTING_blindedSep28.xlsx')
df_blood2_raw = pd.read_csv('cleaned_blood2.csv')

In [25]:
df_static_raw.head() 

,SUBJECT_ID,SUBJECT_KEY,LISTING_DATE,AGE_AT_LISTING,PATIENT_STATUS,PATIENT_STATUS_DESCR,TREATMENT_PHASE,TREATMENT_PHASE_DESCR,TX_NUMBER,SEX,...,BW_4mo,BW_5mo,BW_6mo,BW_7mo,BW_8mo,BW_9mo,BW_10mo,BW_11mo,BW_1yr,BW_>1yr
0,1,1_1,2010-09-24,58.523288,2,Outpatient,5.0,Post-Transplant,1,1,...,1,0,1,1,1,1,1,1,1,1
1,2,2_1,2005-06-15,58.487671,5,Deceased,4.0,Listed,1,1,...,1,0,1,0,1,1,1,1,1,1
2,3,3_1,2006-04-12,66.115068,1,Inactive,4.0,Listed,1,2,...,0,1,1,1,1,1,1,1,1,1
3,4,4_1,2003-07-15,52.309589,1,Inactive,5.0,Post-Transplant,1,2,...,1,0,1,0,1,1,0,1,0,1
4,5,5_1,2016-12-16,65.539726,2,Outpatient,5.0,Post-Transplant,1,1,...,1,1,1,1,1,1,1,1,1,1


In [26]:
df_blood2_raw.head()

,Unnamed: 0,SUBJECT_ID_NA,LISTING_DATE_NA,DATE_OF_LAB_NA,DAYS_FROM_LISTING_NA,avg(NA),avg(HCO3),avg(GLUCOSE),avg(CALCIUM),avg(MG),avg(PHOSPHORUS)
0,10002003-08-26 00:00:002003-06-10 00:00:00,1000.0,8/26/2003,6/10/2003,-77.0,129.0,20.5,4.4,2.22,0.69,0.86
1,10002003-08-26 00:00:002003-09-10 00:00:00,1000.0,8/26/2003,9/10/2003,15.0,137.0,NaN,6.3,NaN,NaN,NaN
2,10002003-08-26 00:00:002003-11-05 00:00:00,1000.0,8/26/2003,11/5/2003,71.0,134.0,23.0,8.6,NaN,NaN,0.92
3,10002003-08-26 00:00:002003-11-11 00:00:00,1000.0,8/26/2003,11/11/2003,77.0,131.0,NaN,8.6,NaN,NaN,NaN
4,10002003-08-26 00:00:002003-12-15 00:00:00,1000.0,8/26/2003,12/15/2003,111.0,133.0,NaN,10.5,NaN,NaN,NaN


In [27]:
df_blood1_raw['SUBJECT_ID'] = df_blood1_raw['SUBJECT_ID'].map(str)

In [28]:
df_blood1_raw.head()

,SUBJECT_ID,LISTING_DATE,DATE_OF_LAB,LAB_INTERVAL,CREAT,BILT,AST,ALT,ALK_PHOS,PLATELETS,WBC,ALB,EST_GFR,MELD,HGB,INR
0,661,2003-02-04,2015-02-04,>1 year,78,11.4,17,21,92,144,8.1,NaN,92,6.0,175,1
1,661,2003-02-04,2015-01-06,>1 year,81,11.9,87,229,122,151,7.3,NaN,88,6.0,173,1
2,661,2003-02-04,2014-08-15,>1 year,77,12.1,18,22,93,139,8.1,NaN,93,9.0,179,1.3
3,661,2003-02-04,2013-08-19,>1 year,77,16,19,22,98,149,8.8,43,NaN,8.0,172,1.12
4,661,2003-02-04,2013-07-03,>1 year,72,9,19,24,107,134,6.6,NaN,101,6.0,161,1


## Data cleaning on blood1 table

In [29]:
# Data cleansing: extract numeric part from cells with mixed values ( 117 norm -> 117)
df_blood1_raw.INR = df_blood1_raw.INR.map(str)
df_blood1_raw.INR = df_blood1_raw.INR.str.extract(r'([-]?[\d.]*)')
df_blood1_raw.CREAT = df_blood1_raw.CREAT.map(str)
df_blood1_raw.CREAT = df_blood1_raw.CREAT.str.extract(r'([-]?[\d.]*)')
df_blood1_raw.BILT = df_blood1_raw.BILT.map(str)
df_blood1_raw.BILT = df_blood1_raw.BILT.str.extract(r'([-]?[\d.]*)')
df_blood1_raw.AST = df_blood1_raw.AST.map(str)
df_blood1_raw.AST = df_blood1_raw.AST.str.extract(r'([-]?[\d.]*)')
df_blood1_raw.ALT = df_blood1_raw.ALT.map(str)
df_blood1_raw.ALT = df_blood1_raw.ALT.str.extract(r'([-]?[\d.]*)')
df_blood1_raw.ALK_PHOS = df_blood1_raw.ALK_PHOS.map(str)
df_blood1_raw.ALK_PHOS = df_blood1_raw.ALK_PHOS.str.extract(r'([-]?[\d.]*)')
df_blood1_raw.PLATELETS = df_blood1_raw.PLATELETS.map(str)
df_blood1_raw.PLATELETS = df_blood1_raw.PLATELETS.str.extract(r'([-]?[\d.]*)')
df_blood1_raw.WBC = df_blood1_raw.WBC.map(str)
df_blood1_raw.WBC = df_blood1_raw.WBC.str.extract(r'([-]?[\d.]*)')
df_blood1_raw.ALB = df_blood1_raw.ALB.map(str)
df_blood1_raw.ALB = df_blood1_raw.ALB.str.extract(r'([-]?[\d.]*)')
df_blood1_raw.EST_GFR = df_blood1_raw.EST_GFR.map(str)
df_blood1_raw.EST_GFR = df_blood1_raw.EST_GFR.str.extract(r'([-]?[\d.]*)')
df_blood1_raw.MELD = df_blood1_raw.MELD.map(str)
df_blood1_raw.MELD = df_blood1_raw.MELD.str.extract(r'([-]?[\d.]*)')
df_blood1_raw.HGB = df_blood1_raw.HGB.map(str)
df_blood1_raw.HGB = df_blood1_raw.HGB.str.extract(r'([-]?[\d.]*)')


In [30]:
df_blood1_raw.head()

,SUBJECT_ID,LISTING_DATE,DATE_OF_LAB,LAB_INTERVAL,CREAT,BILT,AST,ALT,ALK_PHOS,PLATELETS,WBC,ALB,EST_GFR,MELD,HGB,INR
0,661,2003-02-04,2015-02-04,>1 year,78,11.4,17,21,92,144,8.1,,92,6.0,175,1
1,661,2003-02-04,2015-01-06,>1 year,81,11.9,87,229,122,151,7.3,,88,6.0,173,1
2,661,2003-02-04,2014-08-15,>1 year,77,12.1,18,22,93,139,8.1,,93,9.0,179,1.3
3,661,2003-02-04,2013-08-19,>1 year,77,16,19,22,98,149,8.8,43,,8.0,172,1.12
4,661,2003-02-04,2013-07-03,>1 year,72,9,19,24,107,134,6.6,,101,6.0,161,1


In [ ]:
df_static_raw['UNLISTED_REASON']

## Combine two blood tables 

In [32]:
#create identifier to join blood tables 
df_blood1_raw["lab_id"] = df_blood1_raw["SUBJECT_ID"].map(str) + df_blood1_raw["LISTING_DATE"].map(str) + df_blood1_raw["DATE_OF_LAB"].map(str)

In [33]:
# rename the columns for the blood2 table 
df_blood2_raw.columns = ['lab_id', 'SUBJECT_ID_NA', 'LISITNG_DATE','DATE_OF_LAB_NA','DAYS_FROM_LISTING','Na','HCO3','GLUCOSE','CALCIUM','MG','PHOS']

In [34]:
df_blood_raw = pd.DataFrame()

In [35]:
# join table blood1 and blood 2
df_blood_raw = df_blood1_raw.merge(df_blood2_raw, left_on='lab_id', right_on='lab_id',how='outer')


In [37]:
#df_blood_raw.to_csv('raw_blood.csv')
df_blood_raw.head()
df_blood_raw.shape

(303305, 27)

In [38]:
# Preparing joining key for blood and static table 
df_static_raw['LIST_ID'] =  df_static_raw['SUBJECT_ID'].map(str)+df_static_raw['LISTING_DATE'].map(str)
df_blood_raw['LIST_ID'] =  df_blood_raw['SUBJECT_ID'].map(str)+df_blood_raw['LISTING_DATE'].map(str)

In [39]:
#df_blood_cleaned.to_csv('cleaned_blood.csv')
#df_static_raw.to_csv('cleaned_static.csv')

In [40]:
# transfer some columns into string in order to generate sql tables 
df_static_raw['PREV_SURG1'] = df_static_raw['PREV_SURG1'].map(str)
df_static_raw['CAUSE_OF_DEATH_DESCR'] = df_static_raw['CAUSE_OF_DEATH_DESCR'].map(str)
df_static_raw['TREATMENT_PHASE_DESCR'] = df_static_raw['TREATMENT_PHASE_DESCR'].map(str)
df_static_raw['DIALYSIS_TYPE_DESCR'] = df_static_raw['DIALYSIS_TYPE_DESCR'].map(str)
df_static_raw['DIALYSIS_START_DATE'] = df_static_raw['DIALYSIS_START_DATE'].map(str)
df_static_raw['RH_FACTOR_DESCR'] = df_static_raw['RH_FACTOR_DESCR'].map(str)
df_static_raw['SEX_DESCR'] = df_static_raw['SEX_DESCR'].map(str)
df_static_raw['PATIENT_STATUS_DESCR'] = df_static_raw['PATIENT_STATUS_DESCR'].map(str)
df_static_raw['WEIGHT_AT_LISTING'][df_static_raw['WEIGHT_AT_LISTING'] == '76..2'] = 76.2
pd.to_numeric(df_static_raw['WEIGHT_AT_LISTING'])
pd.to_numeric(df_static_raw['HEIGHT_AT_LISTING'])


/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


0       178.00
1          NaN
2       149.00
3          NaN
4       165.00
5       171.00
6          NaN
7          NaN
8       175.00
9          NaN
10      175.00
11      175.00
12      179.00
13      171.00
14         NaN
15         NaN
16      168.00
17      155.00
18      171.00
19         NaN
20         NaN
21      181.00
22      182.00
23      156.00
24         NaN
25      167.64
26         NaN
27      193.00
28         NaN
29      169.00
         ...  
3361       NaN
3362       NaN
3363       NaN
3364       NaN
3365       NaN
3366    161.00
3367    154.00
3368       NaN
3369    178.00
3370    172.00
3371    170.00
3372    172.72
3373    191.00
3374    175.00
3375       NaN
3376       NaN
3377    181.00
3378    173.00
3379    167.00
3380       NaN
3381    150.00
3382    177.00
3383    170.00
3384    172.00
3385    166.00
3386       NaN
3387    170.00
3388       NaN
3389       NaN
3390       NaN
Name: HEIGHT_AT_LISTING, Length: 3391, dtype: float64

## Combine static and blood tables 

In [41]:
# Merge the blood table with the static table
left = df_static_raw
right = df_blood_raw
df_combined_raw = pd.merge(left, right, on='LIST_ID')

In [42]:
list(df_combined_raw)
df_combined_raw.shape

(274867, 88)

In [43]:
df_combined_raw.to_csv('combined_raw.csv')

## Drop and rename colcumns

In [17]:
df_testing = pd.read_csv('combined_raw.csv')

/home/jupyterlab/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (28,66,67,68,69,70,72,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
df_testing.shape

(274867, 89)

In [19]:

df_testing['CREAT'] = pd.to_numeric(df_testing['CREAT'], errors='coerce')
df_testing['CREAT'].dtypes

dtype('float64')

In [20]:
list(df_testing)

['Unnamed: 0',
 'SUBJECT_ID_x',
 'SUBJECT_KEY',
 'LISTING_DATE_x',
 'AGE_AT_LISTING',
 'PATIENT_STATUS',
 'PATIENT_STATUS_DESCR',
 'TREATMENT_PHASE',
 'TREATMENT_PHASE_DESCR',
 'TX_NUMBER',
 'SEX',
 'SEX_DESCR',
 'RACE',
 'RACE_DESCR',
 'HEIGHT_AT_LISTING',
 'WEIGHT_AT_LISTING',
 'BLOOD_TYPE',
 'BLOOD_TYPE_DESCR',
 'RH_FACTOR',
 'RH_FACTOR_DESCR',
 'P_DIAG',
 'P_DIAG_DESCR',
 'SEC_DIAG',
 'SEC_DIAG_DESCR',
 'SMOKER',
 'PREV_SURG1',
 'DIALYSIS_TYPE',
 'DIALYSIS_TYPE_DESCR',
 'DIALYSIS_START_DATE',
 'LIFE_SUPPORT',
 'VENTILATOR_DEPENDENT',
 'IV_INATROPES',
 'VAD',
 'IABP',
 'TIPS',
 'ENCEPHALOPATHY',
 'VARICES',
 'ASCITES',
 'UNLISTED_REASON',
 'UNLISTED_REASON_DESCR',
 'DATE_OF_TRANSPLANT',
 'UNLISTED_DATE',
 'AGE_AT_UNLISTING',
 'AGE_AT_DEATH',
 'CAUSE_OF_DEATH',
 'CAUSE_OF_DEATH_DESCR',
 'Unnamed: 45',
 'BW At Listing',
 'BW_1mo',
 'BW_2mo',
 'BW_3mo',
 'BW_4mo',
 'BW_5mo',
 'BW_6mo',
 'BW_7mo',
 'BW_8mo',
 'BW_9mo',
 'BW_10mo',
 'BW_11mo',
 'BW_1yr',
 'BW_>1yr',
 'LIST_ID',
 'SUBJECT

In [21]:
df_testing.drop(['Unnamed: 45','SUBJECT_ID_y','LISTING_DATE_y','SUBJECT_ID_NA','LISITNG_DATE','DATE_OF_LAB_NA'],axis=1,inplace = True)
df_testing.rename(columns={"CALCIUM": "CALC"}, inplace=True)
df_testing.rename(columns={"PREV_SURG1": "PREV_SURG"}, inplace=True)

In [22]:
df_testing.rename(columns={"SUBJECT_ID_x": "SUBJECT_ID","LISTING_DATE_x":"LISTING_DATE"}, inplace=True)

In [23]:
df_testing['BILT'] = pd.to_numeric(df_testing['BILT'], errors='coerce')
df_testing['AST'] = pd.to_numeric(df_testing['AST'], errors='coerce')
df_testing['ALT'] = pd.to_numeric(df_testing['ALT'], errors='coerce')
df_testing['ALK_PHOS'] = pd.to_numeric(df_testing['ALK_PHOS'], errors='coerce')
df_testing['PLATELETS'] = pd.to_numeric(df_testing['PLATELETS'], errors='coerce')
df_testing['WBC'] = pd.to_numeric(df_testing['WBC'], errors='coerce')
df_testing['ALB'] = pd.to_numeric(df_testing['ALB'], errors='coerce')
df_testing['HGB'] = pd.to_numeric(df_testing['HGB'], errors='coerce')
df_testing['INR'] = pd.to_numeric(df_testing['INR'], errors='coerce')
df_testing['CREAT'] = pd.to_numeric(df_testing['CREAT'], errors='coerce')

In [24]:
df_testing.shape

(274867, 83)

In [25]:
df_testing['Drop_off'] = 0
df_testing.loc[(df_testing.UNLISTED_REASON == 6) | (df_testing.UNLISTED_REASON == 9) | (df_testing.UNLISTED_REASON == 2) | (df_testing.UNLISTED_REASON == 10) | (df_testing.UNLISTED_REASON == 0), 
               'Drop_off'] = 1  

#6 Medical unsuitable
#9 Other 
#2 Patients expired
#10 Status change
#0 Unknown


In [26]:
df_testing['UNLISTED_DATE'] = pd.to_datetime(df_testing['UNLISTED_DATE'])
df_testing['DATE_OF_LAB'] = pd.to_datetime(df_testing['DATE_OF_LAB'])
df_testing['Interval'] = df_testing['UNLISTED_DATE'] - df_testing['DATE_OF_LAB']
df_testing['Interval'] = df_testing['Interval'].dt.days

In [ ]:
df_testing

In [39]:
df_testing['Drop_3M'] = 0
df_testing.loc[(df_testing.Interval <= 90)&(df_testing.Drop_off == 1), 'Drop_3M'] = 1 

In [36]:
df_testing.shape

(274867, 86)

In [38]:
df_positive = df_testing[df_testing.Interval > 0]
df_positive.shape

(58440, 86)

In [40]:
df_testing.to_csv('combined_2.csv')

In [31]:
df_testing.to_excel('combined_2.xlsx')